# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Kociemba's two-phase algo (ALL 3x3x3 cubes) w/ Greedy Baseline Improvement</div>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [INFO](#0)
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
  * [FUNCTIONS FROM WRROSA](#3.0)
  * [FUNCTIONS FROM CRODOC](#3.1)
* [CONVERSIONS](#4)
* [SOLVE](#5)
  * [SCORE](#5.1)
  * [SUBMISSION](#5.2)

<a id="0"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >INFO<br><div> 
    
I've made the necessary translation to puzzles 130-149 so @wrrosa's code works on all 3x3x3 cubes. Additionally, I have added the solutions ensemble compiled by @mansour.
    
Code modified from: 
* https://www.kaggle.com/code/crodoc/1-187-898-greedy-baseline-improvement
* https://www.kaggle.com/code/wrrosa/santa-2023-kociemba-s-two-phase-algo-1-116-550/

<a id="1"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >IMPORTS<br><div> 
    
* https://www.kaggle.com/code/metric/santa-2023-metric
* https://www.kaggle.com/code/jazivxt/wayfinder
* https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards/

In [ ]:
! pip install kociemba > /dev/null
import kociemba

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sympy.combinatorics import Permutation

<a id="2"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 
    
It is becoming difficult to track down the source of some solutions due to all of the ensemble notebooks in use. I've attempted to locate the sources for the files that give the best solution. The Visualize Using Pythreejs appears to use some solutions from a previous version of this notebook: https://www.kaggle.com/code/zaburo/iterative-replacement-of-k-successive-moves
    
Ensemble sources:

-https://www.kaggle.com/code/yunsuxiaozi/k-step-optimal-solution-santa
-https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards
-https://www.kaggle.com/code/jazivxt/visualize-using-pythreejs

In [ ]:
sol1 = pd.read_csv('/kaggle/input/k-step-optimal-solution-santa/submission.csv')
sol2 = pd.read_csv('/kaggle/input/bidirectional-brute-force-w-wildcards/submission.csv')
sol3 = pd.read_csv('/kaggle/input/visualize-using-pythreejs/submission.csv')


ens = pd.DataFrame({
    'solution1': sol1['moves'],
    'solution2': sol2['moves'],
    'solution3': sol3['moves'],
})

ens['ens_solution'] = ens.apply(lambda moves: min(moves, key=lambda x: len(x.split('.'))), axis=1)
ensemble_score = ens['ens_solution'].str.split('.').apply(lambda x: len(x)).sum()


In [ ]:
p = '/kaggle/input/santa-2023/'
path = pd.read_csv(p + 'puzzles.csv')
info = pd.read_csv(p + 'puzzle_info.csv')
sub = pd.read_csv(p + 'sample_submission.csv')

sub['moves'] = ens['ens_solution']

info['allowed_moves_count'] = info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(path, info, how='left', on='puzzle_type')
paths = pd.merge(paths, sub, how='left', on='id')

In [ ]:
allowed_moves = {}

for _, row in info.iterrows():
    allowed_moves[row['puzzle_type']] = eval(row['allowed_moves'])

<a id="3"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [ ]:
def relabel_3x3x3(id, state):
    # Split the string into individual elements.
    state_list = state.split(';')
    if id in range(140, 150):
        # These puzzles has a straightforward translation
        # For each element, determine the replacement based on the criteria.
        for i, val in enumerate(state_list):
            # Extract the number part from the string like 'N33' -> 33.
            num = int(val[1:])
            # Replace based on the given criteria.
            if 0 <= num <= 8:
                state_list[i] = 'A'
            elif 9 <= num <= 17:
                state_list[i] = 'B'
            elif 18 <= num <= 26:
                state_list[i] = 'C'
            elif 27 <= num <= 35:
                state_list[i] = 'D'
            elif 36 <= num <= 44:
                state_list[i] = 'E'
            elif 45 <= num <= 53:
                state_list[i] = 'F'

    elif id in range(130, 140):
        # edges are unique, and therefore can be identified by examining the 
        # solution state and used for translating the inital state.
        edges = [
            (7, 10), (14, 21), (23, 30), (32, 39),
            (12, 41), (16, 46), (34, 52), (5, 19), 
            (1, 28), (3, 37), (25, 50), (43, 48)
        ]

        translations = {
            "AC": "FB",
            "AD": "FC",
            "AE": "FD",
            "AF": "FE",
    
            "BC": "AB",
            "BD": "AC",
            "BF": "AE",
            "BE": "AD",
    
            "CD": "BC",
            "CF": "BE",
            "DE": "CD",
            "EF": "DE",
            
        }
        
        for e1, e2 in edges:
            c1 = state_list[e1]
            c2 = state_list[e2]
            # Sort the strings
            sorted_strings = sorted([c1, c2])
            # Concatenate them
            concatenated_string = ''.join(sorted_strings)
            t1, t2 = translations[concatenated_string]
            if c1 == sorted_strings[0]:
                state_list[e1] = t1
                state_list[e2] = t2
            else:
                state_list[e1] = t2
                state_list[e2] = t1
    return ';'.join(state_list)            
        

<a id="3.0"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">FUNCTIONS FROM WRROSA<br><div>

### Convert state to UBL 
Note that competition data is different than standard notation. Function `state2ubl()` performs two steps: mapping centre to UBL and then converts string into standard notation. Based on:
* https://github.com/hkociemba/RubiksCube-TwophaseSolver/blob/master/enums.py (UBL)
* https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023

In [ ]:
U = ['U', 'F', 'R', 'B', 'L', 'D']

def state2ubl(state):

    dct = {}
    for u in range(len(U)):
        dct[state.split(';')[4+u*9]] = U[u]

    s = ''.join([dct[f] for f in state.split(';')])
    
    return s[:9] + s[18:27] + s[9:18] + s[45:] + s[36:45] + s[27:36]

<a id="3.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">FUNCTIONS FROM CRODOC<br><div>

* https://www.kaggle.com/code/crodoc/1-187-898-greedy-baseline-improvement

In [ ]:
def move_state(state, move, moves_pool):
    
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state

def solve_puzzle(puzzle_id, moves_str):
    # paths.loc[paths['id'] == puzzle_id]
    # todo: assumes index aligned with puzzle id. Need to update.
    current_puzzle = path.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
#     moves = submission_df.loc[puzzle_id]['moves'].split('.')
    moves = moves_str.split('.')
    moves_pool = allowed_moves[puzzle_type]
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # ignore last state
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx]
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('PUZZLE_ID:', puzzle_id)
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

<a id="4"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >CONVERSIONS<br><div> 
    
* https://www.kaggle.com/code/paulorzp/magic-cube-utilities

In [ ]:
moves = eval(info.loc[info['puzzle_type'] == 'cube_3/3/3']['allowed_moves'].values[0])
for move in list(moves):
    moves['-'+move] = np.argsort(moves[move]).tolist()

M = {}
M["U"] = '-d2'
M["R"] = "r0"
M["B"] = "-f2"
M["F"] = "f0"
M["L"] = "-r2"
M["D"] = "d0"
for m in list(M):
    M[m+"2"] = M[m] + '.' + M[m]
    if "-" in M[m]:
        M[m+"'"] = M[m].replace("-","")
    else:
        M[m+"'"] = "-"+M[m]

<a id="5"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [ ]:
for i in paths.loc[paths['puzzle_type'] == 'cube_3/3/3'].iterrows():
    id = i[1]['id']
    cur_state = i[1]['initial_state']
    sol_state = i[1]['solution_state']

    if id >= 130 and id <= 149:
        cur_state = relabel_3x3x3(id, cur_state)
        sol_state = relabel_3x3x3(id, sol_state)


    sol = kociemba.solve(state2ubl(cur_state))

    new_state = cur_state
    mmoves = '.'.join([M[m] for m in sol.split(' ')])

    for move in mmoves.split('.'):
        new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        temp_state = new_state
        if len(init_moves) > 0:
            for move in init_moves.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

        if temp_state == sol_state:
            print(f'solved id: {id}')
            if len(init_moves) > 0:
                mmoves += '.' + init_moves
            if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                # reduce moves
                mmoves_greedy = solve_puzzle(id, mmoves)
                print(f"improved: new length {len(mmoves_greedy.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")                    
                paths.iloc[id,7] = mmoves_greedy
            break

<a id="5.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SCORE<br><div>

* https://www.kaggle.com/code/metric/santa-2023-metric

In [ ]:
print(f"Score: {paths['moves'].str.split('.').apply(lambda x: len(x)).sum()}")

<a id="5.2"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SUBMISSION<br><div>

In [ ]:
paths[['id','moves']].to_csv('submission.csv', index=False)
! head -n 3 submission.csv